# Create album

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd

Using TensorFlow backend.


In [2]:
DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [3]:
import butterfly.album

In [4]:
# Choose your responses, predictors, and size for the image
predictor = 'serum_l'
response = 'plasma_l'
pixels = 40
features = 62

In [ ]:
album = butterfly.album.create_album(DF, predictor, pixels)

In [335]:
import butterfly.CNN

In [336]:
#Decide which dataset you want to predict
response = [col for col in DF if col.startswith(response)]
response.append("patientID")
response.append("trimester")    
response_df = DF[response]
#Make sure your B trimesters are properly converted to 4

In [337]:
#Get every combination of patient IDs in pairs of m's
# Function which returns subset or r length from n 
from itertools import combinations 

m = 2
IDs = response_df['patientID'].unique()
IDs = IDs.tolist()
exclude = list(combinations(IDs, m))

In [338]:
#from joblib import Parallel, delayed
#import multiprocessing

#num_cores = multiprocessing.cpu_count()

#results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

In [341]:
XX = album
yy = response_df.drop(['patientID', 'trimester'], axis =1 ).values

In [320]:
#for cv in range(len(exclude)):
#Create your calibration and validation datasets
pt1ex = response_df.index[response_df['patientID'] == exclude[cv][0]].tolist()
pt2ex = response_df.index[response_df['patientID'] == exclude[cv][1]].tolist()
ptex = pt1ex+pt2ex

#Divide in calibration and validation
X_c = np.delete(XX, ptex, 0)
X_c = X_c.reshape((X_c.shape[0], X_c.shape[1], pixels))
y_c = np.delete(yy, ptex, 0)
y_c = pd.DataFrame(StandardScaler().fit_transform(y_c))

X_v = np.asarray([XX[i]  for i in ptex])
X_v = X_v.reshape((X_v.shape[0], X_v.shape[1], pixels))
y_v = np.asarray([yy[i] for i in ptex])
y_v = pd.DataFrame(StandardScaler().fit_transform(y_v))

In [321]:
#Create your CNN
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(pixels, pixels)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(features))
model.compile(optimizer='adam', loss='mse')

In [322]:
model.fit(X_c, y_c, epochs=3000, verbose=0)

In [283]:
    # demonstrate prediction
   # x_input = X_v
  #  x_input = x_input.reshape((1, pixels, pixels))        
 #   y_pred = model.predict(x_input, verbose=0)


#    cv.append()

In [323]:
# demonstrate prediction
y_pred = model.predict(X_v, verbose = 0)

In [324]:
y_pred = pd.DataFrame(y_pred)

In [325]:
diff = y_v-y_pred

In [326]:
np.set_printoptions(suppress=True)
print(diff)

         0         1         2         3         4         5         6   \
0 -1.189997  0.282872 -0.206189  0.201836  1.140724 -0.648441  0.065761   
1 -1.022549 -0.512389 -1.074073 -0.009745  0.318854 -0.835739  0.314445   
2 -0.766761 -1.551895 -0.033070 -0.286972 -0.523864 -0.876643  0.782067   
3  1.908640  3.124434  3.425289  3.126978  3.419892  2.718077  1.833939   
4  0.931456  0.666289 -0.671044  0.830256  0.106642  1.078831 -1.180568   
5 -0.021089  0.336526 -1.065506 -0.209681 -0.667245  0.885574 -1.074900   
6  0.004648 -1.282011  0.147353 -1.017366 -1.437006 -0.356602 -0.656882   
7  1.197877 -1.313982 -1.015403 -1.206323 -0.800987 -0.394968  0.161126   

         7         8         9   ...        52        53        54        55  \
0 -0.177157 -0.368692 -0.758671  ... -0.794712  2.098785  0.388825  0.504933   
1  0.281636 -0.349362 -0.956931  ... -0.571744  1.013463 -0.416039 -0.810022   
2  0.554702  0.142680 -1.283361  ... -1.127544 -0.412275 -1.355323 -0.862101   
3  1

In [327]:
corr = y_v.corrwith(y_pred, axis = 0).values

In [328]:
np.mean(corr.clip(min=0))

0.12096306039079661